This is a notebook where we try every code by ourselves and write somoe notes in the form of comments 

In [2]:
import os
from dotenv import load_dotenv

load_dotenv

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

<!-- # we have learnt loading spliiting embedding and vector db 
# storing along with query searching with similarity search 
# and retirever now wwe will start building apps -->

We have learnt loading splitting embeddig and storing in vector db then we learnt similaity search and retreiver function and now we will build genai apps

In [6]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x00000213E2A489D0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000213E2A54EE0> root_client=<openai.OpenAI object at 0x00000213E2A49D80> root_async_client=<openai.AsyncOpenAI object at 0x00000213E2A497B0> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


In [7]:
# input and get response from llm

result=llm.invoke("What is generative AI")
print(result)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [9]:
### prompt template 

from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","Hey you are an expert ai engineer. Provide me answers based on the questions "),
        ("user","{input}")
    ]
)

prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hey you are an expert ai engineer. Provide me answers based on the questions '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [12]:
chain=prompt|llm 

# it make the niput go thorugh prompt template then to llm 

response=chain.invoke({"input":"Can you tell me about Langsmith"})
response

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [14]:
type(response)

# this is an AI message

NameError: name 'response' is not defined

In [16]:
# output parser to display response

# str output parser

from langchain_core.output_parsers import StrOutputParser
output_parser=StrOutputParser()
# here we made this ouput to go thtough 
# to parser and get only stirng result
chain=prompt|llm|output_parser

response=chain.invoke("How are you??")

print(response)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### Creating a simple genai application using langchain

What form of app??

so we have a website which has content and we want to scrap it and divide into chunks splits and then do embedidng then store in db then use llm for answers based on that page
 

In [17]:
import os
from dotenv import load_dotenv

load_dotenv

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [38]:
# data ingestion with web based loader 
from langchain_community.document_loaders import WebBaseLoader

# divide data beacuse there is limit for context size for every llm
from langchain_text_splitters import RecursiveCharacterTextSplitter

# embedding to convert the sentence into vectors
from langchain_community.embeddings import OpenAIEmbeddings

from langchain_community.embeddings import OllamaEmbeddings

# for storing vectors
from langchain_chroma import Chroma 


In [51]:
# create llm model
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-4o")

In [52]:
loader=WebBaseLoader("https://docs.smith.langchain.com/administration/tutorials/manage_spend")
website_data=loader.load()
website_data[0].page_content

'\n\n\n\n\nOptimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationTutorialsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmith\nRecommended ReadingBefore diving into this content, it might be helpful to read the following:\nData Retention Conceptual Docs\nUsage Limiting Conceptual Docs\n\nnoteSome of the features mentioned in this guide are not currently available in Enterprise plan due to its\ncustom nature of billing. If you are o

In [53]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(website_data)
documents[0].page_content

'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationTutorialsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmith\nRecommended ReadingBefore diving into this content, it might be helpful to read the following:\nData Retention Conceptual Docs\nUsage Limiting Conceptual Docs'

In [54]:
embeddings=OpenAIEmbeddings(model="gpt-4o")
embeddings_os=OllamaEmbeddings(model="gemma:2b")
db=Chroma.from_documents(documents=documents,embedding=embeddings_os)

In [55]:
db

In [56]:
# now we can query and get content based on similarity search

query="Langsmith has two usage limits: total traces and extended"
result=db.similarity_search(query)

In [57]:
result[0].page_content

'Understand your current usage\u200b\nThe first step of any optimization process is to understand current usage. LangSmith gives two ways to do this: Usage Graph\nand Invoices.\nUsage Graph\u200b\nThe usage graph lets us examine how much of each usage based pricing metric we have consumed lately. It does not directly show\nspend (which we will see later on our draft invoice).\nWe can navigate to the Usage Graph under Settings -> Usage and Billing -> Usage Graph.\n\nWe see in the graph above that there are two usage metrics that LangSmith charges for:\n\nLangSmith Traces (Base Charge)\nLangSmith Traces (Extended Data Retention Upgrades).'

In [61]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002138885AC80>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021388BB1300>, root_client=<openai.OpenAI object at 0x0000021388859B40>, root_async_client=<openai.AsyncOpenAI object at 0x000002138885B940>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, confi

In [65]:
document_chain.invoke({
    "input":"LangSmith has two usage limits: Total traces and exetnded",
    "context":[document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

NameError: name 'document' is not defined

In [73]:
# rewtrieiver

retreiver=db_2.as_retriever()

NameError: name 'db_2' is not defined

In [67]:
from langchain.chains import create_retrieval_chain

In [70]:
r_chain=create_retrieval_chain(retreiver,document_chain)
r_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000021388BB1630>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | ChatOpe

In [72]:
# get the response from the llm

response=r_chain.invoke({"input":"langsmith has two usage limits :total trces and extended"})
response

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
import os
from dotenv import load_dotenv

from langchain_community.llms import Ollama
import streamlit as st 
load_dotenv()


In [74]:
! pip install streamlit

  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.8 MB ? eta -:--:--
   ------- -------------------------------- 1.8/9.8 MB 8.4 MB/s eta 0:00:01
   ----------------------- ---------------- 5.8/9.8 MB 13.5 MB/s eta 0:00:01
   ---------------------------------------- 9.8/9.8 MB 16.5 MB/s eta 0:00:00
Using cached altair-5.5.0-py3-none-any.whl (731 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using c